In [4]:
import csv
import json
import requests
import math
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as sm
from pandas.tseries.offsets import MonthEnd
from collections import Counter
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

In [5]:
# Get every single token and convert to pd dataframe

url = "https://api.coingecko.com/api/v3/coins/list"

headers = {"x-cg-pro-api-key": "CG-UHjW13FD2AW8Wxd4DiaJKJ1H"}

response = requests.get(url, headers=headers)
response_list = json.loads(response.text)

all_coins = pd.DataFrame(response_list)
all_coins['name'] = all_coins['name'].str.lower()
#print(all_coins)

In [6]:
# Get all token categories and convert names to id

coins = dict()
updated_coins = dict()

with open('token-categories.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        coins[row[0].strip().lower()] = row[1:]

coins['bitcoin'] = coins['\ufeffbitcoin']
del coins['\ufeffbitcoin']
#print(coins)

for coin in coins:
    row = all_coins[all_coins['name'] == coin]
    if row.empty:
        row = all_coins[all_coins['id'] == coin]

    id = row.iloc[0]['id']
    updated_coins[id] = coins[coin]
    #print(id, coin)

#print(updated_coins)
#print(coins['ordi'])


In [7]:
# Get all coins from top100 list and construct dataframe of those with at least 1 appearance
# 247 coins that have had at least one appearance in the top 100

top100_df = pd.read_excel('top100.xlsx')
top100_df = top100_df.set_index('date_rebal_Q')
sums = top100_df.sum()
zerosum = sums[sums == 0].index

top100_df = top100_df.drop(zerosum, axis=1)
#print(top100_df)

In [8]:
categories = dict(set())
other = []

for coin in top100_df.columns:
    if coin in updated_coins:
        if updated_coins[coin][1] == 'Memecoin':
            if 'Memecoin' in categories:
                categories['Memecoin'].add(coin)
            else:
                categories['Memecoin'] = {coin}
        else:
            if updated_coins[coin][0] in categories:
                categories[updated_coins[coin][0]].add(coin)
            else:
                categories[updated_coins[coin][0]] = {coin}
    else:
        if 'n.a.' in categories:
            categories['n.a.'].add(coin)
        else:
            categories['n.a.'] = {coin}

#convert categories to csv
max_len = max(len(arr) for arr in categories.values())
categories_list = {key: list(value) + [None] * (max_len - len(value)) for key, value in categories.items()}
categories_df = pd.DataFrame(categories_list)
categories_df.to_csv('top100_categories.csv')

#print(categories)
#print(categories_df)

In [9]:
# Create list of all coins
top100 = set()
for category in categories.values():
    top100 = top100.union(category)

#print(top100)

In [10]:
# Create df for all and each catagory (10 categories + 1 for All)

all_coin_price = pd.read_csv('prices.csv')
all_coin_price.rename(columns={all_coin_price.columns[0] : 'Date'}, inplace=True)
all_coin_price['Date'] = pd.to_datetime(all_coin_price['Date'])
all_coin_price.set_index('Date', inplace=True)

top_coins_data = all_coin_price[all_coin_price.columns.intersection(top100)]

cat_dfs = {}
for category, values in categories.items():
    cat_dfs[category] = top_coins_data[top_coins_data.columns.intersection(values)]

cat_dfs['All'] = top_coins_data

# convert top coins to csv
top_coins_data.to_csv('top100_price_data.csv')

all_wo_mc = top_coins_data
for col in top_coins_data.columns:
    if col in cat_dfs['Memecoin'].columns:
        all_wo_mc = all_wo_mc.drop(columns=[col], axis=1)

cat_dfs['AllwoMC'] = all_wo_mc

#print(all_wo_mc)
#print(cat_dfs['Memecoin'])
#print(top_coins_data)

In [11]:
# Get weekly pct change for last, median, mean

last_weekly = {}
median_weekly = {}
mean_weekly = {}

for category, df in cat_dfs.items():

    last = df[df.index >= '2019-12-29'].resample('W').last().pct_change() * 100
    median = df[df.index >= '2019-12-29'].resample('W').median().pct_change() * 100
    mean = df[df.index >= '2019-12-29'].resample('W').mean().pct_change() * 100

    last = last[last.index.year >= 2020]
    median = median[median.index.year >= 2020]
    mean = mean[mean.index.year >= 2020]

    last_weekly[category] = last
    median_weekly[category] = median
    mean_weekly[category] = mean

#print(last_weekly['Memecoin'])
#print(median_weekly['Memecoin'])
#print(mean_weekly['Memecoin'])

In [12]:
# Get monthly pct change for last, median, mean

last_monthly = {}
median_monthly = {}
mean_monthly = {}

for category, df in cat_dfs.items():

    last = df[df.index >= '2019-12-29'].resample('M').last().pct_change() * 100
    median = df[df.index >= '2019-12-29'].resample('M').median().pct_change() * 100
    mean = df[df.index >= '2019-12-29'].resample('M').mean().pct_change() * 100

    last = last[last.index.year >= 2020]
    median = median[median.index.year >= 2020]
    mean = mean[mean.index.year >= 2020]

    last_monthly[category] = last
    median_monthly[category] = median
    mean_monthly[category] = mean

#print(last_monthly['Memecoin'])
#print(median_monthly['Memecoin'])
#print(mean_monthly['Memecoin'])

In [13]:
# Get mean/median for each category -> weekly

last_weekly_comb = {}
median_weekly_comb = {}
mean_weekly_comb = {}

for category, df in last_weekly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    last_weekly_comb[category] = combined_df

for category, df in median_weekly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    median_weekly_comb[category] = combined_df

for category, df in mean_weekly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    mean_weekly_comb[category] = combined_df


#print(last_weekly_comb['Memecoin'])
#print(median_weekly_comb['Memecoin'])
#print(mean_weekly_comb['Memecoin'])

In [14]:
# Get mean/median for each category -> monthly

last_monthly_comb = {}
median_monthly_comb = {}
mean_monthly_comb = {}

for category, df in last_monthly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    last_monthly_comb[category] = combined_df

for category, df in median_monthly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    median_monthly_comb[category] = combined_df

for category, df in mean_monthly.items():
    row_means = df.mean(axis=1)
    row_medians = df.median(axis=1)

    combined_df = pd.DataFrame({'Mean': row_means, 'Median': row_medians})

    mean_monthly_comb[category] = combined_df

#print(last_monthly_comb['All'])
#print(median_monthly_comb['Memecoin'])
print(mean_monthly_comb['AllwoMC'])

                  Mean     Median
Date                             
2020-01-31   11.149210   8.741961
2020-02-29   37.910122  35.262697
2020-03-31  -31.223743 -35.262895
2020-04-30   -1.687409  -5.168269
2020-05-31   31.741330  18.827956
2020-06-30   28.473237  18.734174
2020-07-31   23.891363   6.873170
2020-08-31   60.181879  33.590104
2020-09-30   -5.370262 -16.929585
2020-10-31  -13.777548 -15.199791
2020-11-30    1.541230  -1.356534
2020-12-31   17.792091  10.646913
2021-01-31   39.401216  24.344157
2021-02-28  140.043848  98.663957
2021-03-31   73.195547  41.076597
2021-04-30   52.311585  32.001970
2021-05-31   -3.183613 -14.299318
2021-06-30  -38.106079 -40.512733
2021-07-31  -11.300368 -16.361813
2021-08-31   51.029739  42.191086
2021-09-30   20.779058   3.013009
2021-10-31   10.848411  -1.547224
2021-11-30   28.925742   8.032919
2021-12-31  -18.049817 -22.918939
2022-01-31  -15.921295 -19.985816
2022-02-28  -16.896483 -18.497373
2022-03-31   -2.631240  -5.872211
2022-04-30    

In [15]:
# @title
# find weeks where MC outperformed market

df_all = last_weekly_comb['All']
df_meme = last_weekly_comb['Memecoin']
last_w_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
last_w_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

df_all = median_weekly_comb['All']
df_meme = median_weekly_comb['Memecoin']
median_w_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
median_w_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

df_all = mean_weekly_comb['All']
df_meme = mean_weekly_comb['Memecoin']
mean_w_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
mean_w_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

print(median_w_avg)

                 Mean
Date                 
2020-01-05  -1.098211
2020-01-12   3.942846
2020-01-26  -0.131479
2020-03-01 -13.216326
2020-03-15 -15.055490
...               ...
2023-12-24   3.507885
2024-01-14  -2.422849
2024-02-18  10.287053
2024-03-03  11.643690
2024-03-10  15.149879

[97 rows x 1 columns]


In [16]:
# @title
# find months where MC outperformed market

df_all = last_monthly_comb['All']
df_meme = last_monthly_comb['Memecoin']
last_m_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
last_m_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

df_all = median_monthly_comb['All']
df_meme = median_monthly_comb['Memecoin']
median_m_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
median_m_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

df_all = mean_monthly_comb['All']
df_meme = mean_monthly_comb['Memecoin']
mean_m_avg = df_all[df_meme['Mean'] > df_all['Mean']]['Mean'].to_frame()
mean_m_med = df_all[df_meme['Median'] > df_all['Median']]['Median'].to_frame()

print(median_m_avg)

                  Mean
Date                  
2020-03-31  -39.356719
2020-07-31   21.889810
2020-10-31  -11.521961
2021-01-31   40.052885
2021-02-28  167.764018
2021-03-31   84.496346
2021-04-30   59.305166
2021-05-31   21.937481
2021-06-30  -39.220624
2021-07-31  -10.902123
2021-10-31   11.887622
2021-11-30   32.917036
2022-01-31   -9.268217
2022-02-28  -18.958761
2022-05-31  -47.015844
2022-06-30  -22.512403
2022-09-30  -13.529852
2022-10-31   -5.620179
2022-11-30  -14.683361
2023-01-31   20.046657
2023-02-28   27.770374
2023-04-30    2.775594
2023-05-31  -14.599699
2023-11-30   43.847381
2023-12-31   28.438163
2024-03-31   36.834583


In [17]:
# @title
# Convert rows of prices to lists for linear regression

#last, avg, weekly
lwa_ret_all = []
lwa_ret_meme = []
lwa_same_all = []
lwa_ret_awoc = []
lwa_next_awoc = []

for row in last_w_avg.iterrows():
    lwa_ret_meme.append(last_weekly_comb['Memecoin'].loc[row[0]]['Mean'])
    lwa_ret_awoc.append(last_weekly_comb['AllwoMC'].loc[row[0]]['Mean'])

for date in last_w_avg.index.tolist():
    try:
        next_week = last_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        same_week = last_weekly_comb['All'].loc[date]['Mean']
        next_awoc = last_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        lwa_ret_all.append(next_week)
        lwa_same_all.append(same_week)
        lwa_next_awoc.append(next_awoc)
    except KeyError:
        continue

lwa_ret_meme = lwa_ret_meme[:-1]
lwa_ret_awoc = lwa_ret_awoc[:-1]
lwa_ret_all = np.array(lwa_ret_all)
lwa_ret_meme = np.array(lwa_ret_meme)
lwa_same_all = np.array(lwa_same_all)
lwa_ret_awoc = np.array(lwa_ret_awoc)
lwa_next_awoc = np.array(lwa_next_awoc)

r_val, p_val = pearsonr(lwa_ret_meme, lwa_ret_all)
print(f'last, avg, weekly-- r val: {r_val}, p val: {p_val}')


#last, med, weekly
lwm_ret_all = []
lwm_ret_meme = []
lwm_same_all = []
lwm_ret_awoc = []
lwm_next_awoc = []

for row in last_w_med.iterrows():
    lwm_ret_meme.append(last_weekly_comb['Memecoin'].loc[row[0]]['Median'])
    lwm_ret_awoc.append(last_weekly_comb['AllwoMC'].loc[row[0]]['Median'])

for date in last_w_med.index.tolist():
    try:
        next_week = last_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        same_week = last_weekly_comb['All'].loc[date]['Median']
        next_awoc = last_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        lwm_ret_all.append(next_week)
        lwm_same_all.append(same_week)
        lwm_next_awoc.append(next_awoc)
    except KeyError:
        continue

lwm_ret_meme = lwm_ret_meme[:-1]
lwm_ret_awoc = lwm_ret_awoc[:-1]
lwm_ret_all = np.array(lwm_ret_all)
lwm_ret_meme = np.array(lwm_ret_meme)
lwm_same_all = np.array(lwm_same_all)
lwm_ret_awoc = np.array(lwm_ret_awoc)
lwm_next_awoc = np.array(lwm_next_awoc)

r_val, p_val = pearsonr(lwm_ret_meme, lwm_ret_all)
print(f'last, med, weekly-- r val: {r_val}, p val: {p_val}')


#median, avg, weekly
mwa_ret_all = []
mwa_ret_meme = []
mwa_same_all = []
mwa_ret_awoc = []
mwa_next_awoc = []

for row in median_w_avg.iterrows():
    mwa_ret_meme.append(median_weekly_comb['Memecoin'].loc[row[0]]['Mean'])
    mwa_ret_awoc.append(median_weekly_comb['AllwoMC'].loc[row[0]]['Mean'])

for date in median_w_avg.index.tolist():
    try:
        next_week = median_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        same_week = median_weekly_comb['All'].loc[date]['Mean']
        next_awoc = median_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        mwa_ret_all.append(next_week)
        mwa_same_all.append(same_week)
        mwa_next_awoc.append(next_awoc)
    except KeyError:
        continue

mwa_ret_meme = mwa_ret_meme[:-1]
mwa_ret_awoc = mwa_ret_awoc[:-1]
mwa_ret_all = np.array(mwa_ret_all)
mwa_ret_meme = np.array(mwa_ret_meme)
mwa_same_all = np.array(mwa_same_all)
mwa_ret_awoc = np.array(mwa_ret_awoc)
mwa_next_awoc = np.array(mwa_next_awoc)

r_val, p_val = pearsonr(mwa_ret_meme, mwa_ret_all)
print(f'median, avg, weekly-- r val: {r_val}, p val: {p_val}')


#median, med, weekly
mwm_ret_all = []
mwm_ret_meme = []
mwm_same_all = []
mwm_ret_awoc = []
mwm_next_awoc = []

for row in median_w_med.iterrows():
    mwm_ret_meme.append(median_weekly_comb['Memecoin'].loc[row[0]]['Median'])
    mwm_ret_awoc.append(median_weekly_comb['AllwoMC'].loc[row[0]]['Median'])


for date in median_w_med.index.tolist():
    try:
        next_week = median_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        same_week = median_weekly_comb['All'].loc[date]['Median']
        next_awoc = median_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        mwm_ret_all.append(next_week)
        mwm_same_all.append(same_week)
        mwm_next_awoc.append(next_awoc)
    except KeyError:
        continue

mwm_ret_meme = mwm_ret_meme[:-1]
mwm_ret_awoc = mwm_ret_awoc[:-1]
mwm_ret_all = np.array(mwm_ret_all)
mwm_ret_meme = np.array(mwm_ret_meme)
mwm_same_all = np.array(mwm_same_all)
mwm_ret_awoc = np.array(mwm_ret_awoc)
mwm_next_awoc = np.array(mwm_next_awoc)

r_val, p_val = pearsonr(mwm_ret_meme, mwm_ret_all)
print(f'median, med, weekly-- r val: {r_val}, p val: {p_val}')


#mean, avg, weekly
awa_ret_all = []
awa_ret_meme = []
awa_same_all = []
awa_ret_awoc = []
awa_next_awoc = []

for row in mean_w_avg.iterrows():
    awa_ret_meme.append(mean_weekly_comb['Memecoin'].loc[row[0]]['Mean'])
    awa_ret_awoc.append(mean_weekly_comb['AllwoMC'].loc[row[0]]['Mean'])


for date in mean_w_avg.index.tolist():
    try:
        next_week = mean_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        same_week = mean_weekly_comb['All'].loc[date]['Mean']
        next_awoc = mean_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Mean']
        awa_ret_all.append(next_week)
        awa_same_all.append(same_week)
        awa_next_awoc.append(next_awoc)
    except KeyError:
        continue

awa_ret_meme = awa_ret_meme[:-1]
awa_ret_awoc = awa_ret_awoc[:-1]
awa_ret_all = np.array(awa_ret_all)
awa_ret_meme = np.array(awa_ret_meme)
awa_same_all = np.array(awa_same_all)
awa_ret_awoc = np.array(awa_ret_awoc)
awa_next_awoc = np.array(awa_next_awoc)

r_val, p_val = pearsonr(awa_ret_meme, awa_ret_all)
print(f'mean, avg, weekly-- r val: {r_val}, p val: {p_val}')


#mean, med, weekly
awm_ret_all = []
awm_ret_meme = []
awm_same_all = []
awm_ret_awoc = []
awm_next_awoc = []

for row in mean_w_med.iterrows():
    awm_ret_meme.append(mean_weekly_comb['Memecoin'].loc[row[0]]['Median'])
    awm_ret_awoc.append(mean_weekly_comb['AllwoMC'].loc[row[0]]['Median'])

for date in mean_w_med.index.tolist():
    try:
        next_week = mean_weekly_comb['All'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        same_week = mean_weekly_comb['All'].loc[date]['Median']
        next_awoc = mean_weekly_comb['AllwoMC'].loc[(date + pd.DateOffset(weeks=1))]['Median']
        awm_ret_all.append(next_week)
        awm_same_all.append(same_week)
        awm_next_awoc.append(next_awoc)
    except KeyError:
        continue

awm_ret_meme = awm_ret_meme[:-1]
awm_ret_awoc = awm_ret_awoc[:-1]
awm_ret_all = np.array(awm_ret_all)
awm_ret_meme = np.array(awm_ret_meme)
awm_same_all = np.array(awm_same_all)
awm_ret_awoc = np.array(awm_ret_awoc)
awm_next_awoc = np.array(awm_next_awoc)

r_val, p_val = pearsonr(awm_ret_meme, awm_ret_all)
print(f'mean, med, weekly-- r val: {r_val}, p val: {p_val}')

last, avg, weekly-- r val: 0.02573914109065292, p val: 0.8075824666365163
last, med, weekly-- r val: 0.012877570902992411, p val: 0.8957625684682027
median, avg, weekly-- r val: 0.40079722798482165, p val: 5.197596737525357e-05
median, med, weekly-- r val: 0.3270938769630538, p val: 0.0011425872810769413
mean, avg, weekly-- r val: 0.3019495374915806, p val: 0.003823875376534725
mean, med, weekly-- r val: 0.2785664670361525, p val: 0.007500496329899813


In [18]:
# @title
# Convert rows of prices to lists for linear regression

#last, avg, monthly
lma_ret_all = []
lma_ret_meme = []
lma_same_all = []
lma_ret_awoc = []
lma_next_awoc = []

for row in last_m_avg.iterrows():
    lma_ret_meme.append(last_monthly_comb['Memecoin'].loc[row[0]]['Mean'])
    lma_ret_awoc.append(last_monthly_comb['AllwoMC'].loc[row[0]]['Mean'])

for date in last_m_avg.index.tolist():
    try:
        next_month = last_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        same_month = last_monthly_comb['All'].loc[date]['Mean']
        next_awoc = last_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        lma_ret_all.append(next_month)
        lma_same_all.append(same_month)
        lma_next_awoc.append(next_awoc)
    except KeyError:
        continue

lma_ret_meme = lma_ret_meme[:-1]
lma_ret_awoc = lma_ret_awoc[:-1]
lma_ret_all = np.array(lma_ret_all)
lma_ret_meme = np.array(lma_ret_meme)
lma_same_all = np.array(lma_same_all)
lma_ret_awoc = np.array(lma_ret_awoc)
lma_next_awoc = np.array(lma_next_awoc)

r_val, p_val = pearsonr(lma_ret_meme, lma_ret_all)
print(f'last, avg, monthly-- r val: {r_val}, p val: {p_val}')


#last, med, monthly
lmm_ret_all = []
lmm_ret_meme = []
lmm_same_all = []
lmm_ret_awoc = []
lmm_next_awoc = []

for row in last_m_med.iterrows():
    lmm_ret_meme.append(last_monthly_comb['Memecoin'].loc[row[0]]['Median'])
    lmm_ret_awoc.append(last_monthly_comb['AllwoMC'].loc[row[0]]['Median'])

for date in last_m_med.index.tolist():
    try:
        next_month = last_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        same_month = last_monthly_comb['All'].loc[date]['Median']
        next_awoc = last_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        lmm_ret_all.append(next_month)
        lmm_same_all.append(same_month)
        lmm_next_awoc.append(next_awoc)
    except KeyError:
        continue

lmm_ret_meme = lmm_ret_meme[:-1]
lmm_ret_awoc = lmm_ret_awoc[:-1]
lmm_ret_all = np.array(lmm_ret_all)
lmm_ret_meme = np.array(lmm_ret_meme)
lmm_same_all = np.array(lmm_same_all)
lmm_ret_awoc = np.array(lmm_ret_awoc)
lmm_next_awoc = np.array(lmm_next_awoc)

r_val, p_val = pearsonr(lmm_ret_meme, lmm_ret_all)
print(f'last, med, monthly-- r val: {r_val}, p val: {p_val}')


#median, avg, monthly
mma_ret_all = []
mma_ret_meme = []
mma_same_all = []
mma_ret_awoc = []
mma_next_awoc = []


for row in median_m_avg.iterrows():
    mma_ret_meme.append(median_monthly_comb['Memecoin'].loc[row[0]]['Mean'])
    mma_ret_awoc.append(median_monthly_comb['AllwoMC'].loc[row[0]]['Mean'])

for date in median_m_avg.index.tolist():
    try:
        next_month = median_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        same_month = median_monthly_comb['All'].loc[date]['Mean']
        next_awoc = median_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        mma_ret_all.append(next_month)
        mma_same_all.append(same_month)
        mma_next_awoc.append(next_awoc)
    except KeyError:
        continue

mma_ret_meme = mma_ret_meme[:-1]
mma_ret_awoc = mma_ret_awoc[:-1]
mma_ret_all = np.array(mma_ret_all)
mma_ret_meme = np.array(mma_ret_meme)
mma_same_all = np.array(mma_same_all)
mma_ret_awoc = np.array(mma_ret_awoc)
mma_next_awoc = np.array(mma_next_awoc)

r_val, p_val = pearsonr(mma_ret_meme, mma_ret_all)
print(f'median, avg, monthly-- r val: {r_val}, p val: {p_val}')


#median, med, monthly
mmm_ret_all = []
mmm_ret_meme = []
mmm_same_all = []
mmm_ret_awoc = []
mmm_next_awoc = []

for row in median_m_med.iterrows():
    mmm_ret_meme.append(median_monthly_comb['Memecoin'].loc[row[0]]['Median'])
    mmm_ret_awoc.append(median_monthly_comb['AllwoMC'].loc[row[0]]['Median'])

for date in median_m_med.index.tolist():
    try:
        next_month = median_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        same_month = median_monthly_comb['All'].loc[date]['Median']
        next_awoc = median_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        mmm_ret_all.append(next_month)
        mmm_same_all.append(same_month)
        mmm_next_awoc.append(next_awoc)
    except KeyError:
        continue

mmm_ret_meme = mmm_ret_meme[:-1]
mmm_ret_awoc = mmm_ret_awoc[:-1]
mmm_ret_all = np.array(mmm_ret_all)
mmm_ret_meme = np.array(mmm_ret_meme)
mmm_same_all = np.array(mmm_same_all)
mmm_ret_awoc = np.array(mmm_ret_awoc)
mmm_next_awoc = np.array(mmm_next_awoc)

r_val, p_val = pearsonr(mmm_ret_meme, mmm_ret_all)
print(f'median, med, monthly-- r val: {r_val}, p val: {p_val}')


#mean, avg, monthly
ama_ret_all = []
ama_ret_meme = []
ama_same_all = []
ama_ret_awoc = []
ama_next_awoc = []

for row in mean_m_avg.iterrows():
    ama_ret_meme.append(mean_monthly_comb['Memecoin'].loc[row[0]]['Mean'])
    ama_ret_awoc.append(mean_monthly_comb['AllwoMC'].loc[row[0]]['Mean'])

for date in mean_m_avg.index.tolist():
    try:
        next_month = mean_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        same_month = mean_monthly_comb['All'].loc[date]['Mean']
        next_awoc = mean_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Mean']
        ama_ret_all.append(next_month)
        ama_same_all.append(same_month)
        ama_next_awoc.append(next_awoc)
    except KeyError:
        continue

ama_ret_meme = ama_ret_meme[:-1]
ama_ret_awoc = ama_ret_awoc[:-1]
ama_ret_all = np.array(ama_ret_all)
ama_ret_meme = np.array(ama_ret_meme)
ama_same_all = np.array(ama_same_all)
ama_ret_awoc = np.array(ama_ret_awoc)
ama_next_awoc = np.array(ama_next_awoc)

r_val, p_val = pearsonr(ama_ret_meme, ama_ret_all)
print(f'mean, avg, monthly-- r val: {r_val}, p val: {p_val}')


#mean, med, monthly
amm_ret_all = []
amm_ret_meme = []
amm_same_all = []
amm_ret_awoc = []
amm_next_awoc = []

for row in mean_m_med.iterrows():
    amm_ret_meme.append(mean_monthly_comb['Memecoin'].loc[row[0]]['Median'])
    amm_ret_awoc.append(mean_monthly_comb['AllwoMC'].loc[row[0]]['Median'])

for date in mean_m_med.index.tolist():
    try:
        next_month = mean_monthly_comb['All'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        same_month = mean_monthly_comb['All'].loc[date]['Median']
        next_awoc = mean_monthly_comb['AllwoMC'].loc[(date + pd.DateOffset(months=1)) + MonthEnd(0)]['Median']
        amm_ret_all.append(next_month)
        amm_same_all.append(same_month)
        amm_next_awoc.append(next_awoc)
    except KeyError:
        continue

amm_ret_meme = amm_ret_meme[:-1]
amm_ret_awoc = amm_ret_awoc[:-1]
amm_ret_all = np.array(amm_ret_all)
amm_ret_meme = np.array(amm_ret_meme)
amm_same_all = np.array(amm_same_all)
amm_ret_awoc = np.array(amm_ret_awoc)
amm_next_awoc = np.array(amm_next_awoc)

r_val, p_val = pearsonr(amm_ret_meme, amm_ret_all)
print(f'mean, med, monthly-- r val: {r_val}, p val: {p_val}')

last, avg, monthly-- r val: 0.5023888533472182, p val: 0.01048882922132753
last, med, monthly-- r val: 0.3641805548866921, p val: 0.06183044044717421
median, avg, monthly-- r val: 0.19127620662578707, p val: 0.35971168180643215
median, med, monthly-- r val: 0.05105930188538512, p val: 0.8003281245261388
mean, avg, monthly-- r val: 0.25474910133189477, p val: 0.20914085861866552
mean, med, monthly-- r val: 0.09828253685057897, p val: 0.6257556638893881


In [19]:
# @title
# Compare mc top vs next week market
# Last has high p value (>0.8)

df_lwa = pd.DataFrame({'All': lwa_ret_all, 'Memecoin': lwa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lwa).fit()
print(result.params)
print(result.summary())

df_lwm = pd.DataFrame({'All': lwm_ret_all, 'Memecoin': lwm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lwm).fit()
print(result.params)
print(result.summary())


df_mwa = pd.DataFrame({'All': mwa_ret_all, 'Memecoin': mwa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mwa).fit()
print(result.params)
print(result.summary())

df_mwm = pd.DataFrame({'All': mwm_ret_all, 'Memecoin': mwm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mwm).fit()
print(result.params)
print(result.summary())

df_awa = pd.DataFrame({'All': awa_ret_all, 'Memecoin': awa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mwa).fit()
print(result.params)
print(result.summary())

df_awm = pd.DataFrame({'All': awm_ret_all, 'Memecoin': awm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_awm).fit()
print(result.params)
print(result.summary())

Intercept    6.037455
Memecoin     0.003006
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    All   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                   0.05966
Date:                Mon, 25 Mar 2024   Prob (F-statistic):              0.808
Time:                        06:52:10   Log-Likelihood:                -442.88
No. Observations:                  92   AIC:                             889.8
Df Residuals:                      90   BIC:                             894.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

In [ ]:
# @title
# Compare same week market vs next week market
# Last has high p value (>0.8)

df_lwa_same = pd.DataFrame({'Next': lwa_ret_all, 'Same': lwa_same_all})

result = sm.ols(formula='Next ~ Same', data=df_lwa_same).fit()
print(result.params)
print(result.summary())

df_lwm_same = pd.DataFrame({'Next': lwm_ret_all, 'Same': lwm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_lwm_same).fit()
print(result.params)
print(result.summary())


df_mwa_same = pd.DataFrame({'Next': mwa_ret_all, 'Same': mwa_same_all})

result = sm.ols(formula='Next ~ Same', data=df_mwa_same).fit()
print(result.params)
print(result.summary())

df_mwm_same = pd.DataFrame({'Next': mwm_ret_all, 'Same': mwm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_mwm_same).fit()
print(result.params)
print(result.summary())

df_awa_same = pd.DataFrame({'Next': awa_ret_all, 'Same': awa_same_all})

result = sm.ols(formula='Next ~ Same', data=df_awa_same).fit()
print(result.params)
print(result.summary())

df_awm_same = pd.DataFrame({'Next': awm_ret_all, 'Same': awm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_awm_same).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare mc top and market w/o mc vs next week market
# Last has high p value (>0.4)

df_lwa_mc_womc = pd.DataFrame({'All': lwa_ret_all, 'Memecoin': lwa_ret_meme, 'AwoMC': lwa_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_lwa_mc_womc).fit()
print(result.params)
print(result.summary())

df_lwm_mc_womc = pd.DataFrame({'All': lwm_ret_all, 'Memecoin': lwm_ret_meme, 'AwoMC': lwm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_lwm_mc_womc).fit()
print(result.params)
print(result.summary())

df_mwa_mc_womc = pd.DataFrame({'All': mwa_ret_all, 'Memecoin': mwa_ret_meme, 'AwoMC': mwa_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_mwa_mc_womc).fit()
print(result.params)
print(result.summary())

df_mwm_mc_womc = pd.DataFrame({'All': mwm_ret_all, 'Memecoin': mwm_ret_meme, 'AwoMC': mwm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_mwm_mc_womc).fit()
print(result.params)
print(result.summary())

df_awa_mc_womc = pd.DataFrame({'All': awa_ret_all, 'Memecoin': awa_ret_meme, 'AwoMC': awa_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_awa_mc_womc).fit()
print(result.params)
print(result.summary())

df_awm_mc_womc = pd.DataFrame({'All': awm_ret_all, 'Memecoin': awm_ret_meme, 'AwoMC': awm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_awm_mc_womc).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare mc top vs next week market w/o mc
# Last has high p value (>0.8)

df_lwa_womc = pd.DataFrame({'All': lwa_next_awoc, 'Memecoin': lwa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lwa_womc).fit()
print(result.params)
print(result.summary())

df_lwm_womc = pd.DataFrame({'All': lwm_next_awoc, 'Memecoin': lwm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lwm_womc).fit()
print(result.params)
print(result.summary())

df_mwa_womc = pd.DataFrame({'All': mwa_next_awoc, 'Memecoin': mwa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mwa_womc).fit()
print(result.params)
print(result.summary())

df_mwm_womc = pd.DataFrame({'All': mwm_next_awoc, 'Memecoin': mwm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mwm_womc).fit()
print(result.params)
print(result.summary())

df_awa_womc = pd.DataFrame({'All': awa_next_awoc, 'Memecoin': awa_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_awa_womc).fit()
print(result.params)
print(result.summary())

df_awm_womc = pd.DataFrame({'All': awm_next_awoc, 'Memecoin': awm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_awm_womc).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare mc top vs next month market
# Last has high p value (>0.8)

df_lma = pd.DataFrame({'All': lma_ret_all, 'Memecoin': lma_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lma).fit()
print(result.params)
print(result.summary())

df_lmm = pd.DataFrame({'All': lmm_ret_all, 'Memecoin': lmm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lmm).fit()
print(result.params)
print(result.summary())


df_mma = pd.DataFrame({'All': mma_ret_all, 'Memecoin': mma_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mma).fit()
print(result.params)
print(result.summary())

df_mmm = pd.DataFrame({'All': mmm_ret_all, 'Memecoin': mmm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mmm).fit()
print(result.params)
print(result.summary())

df_ama = pd.DataFrame({'All': ama_ret_all, 'Memecoin': ama_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mma).fit()
print(result.params)
print(result.summary())

df_amm = pd.DataFrame({'All': amm_ret_all, 'Memecoin': amm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_amm).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare same week market vs next month market
# Last has high p value (>0.8)

df_lma_same = pd.DataFrame({'Next': lma_ret_all, 'Same': lma_same_all})

result = sm.ols(formula='Next ~ Same', data=df_lma_same).fit()
print(result.params)
print(result.summary())

df_lmm_same = pd.DataFrame({'Next': lmm_ret_all, 'Same': lmm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_lmm_same).fit()
print(result.params)
print(result.summary())


df_mma_same = pd.DataFrame({'Next': mma_ret_all, 'Same': mma_same_all})

result = sm.ols(formula='Next ~ Same', data=df_mma_same).fit()
print(result.params)
print(result.summary())

df_mmm_same = pd.DataFrame({'Next': mmm_ret_all, 'Same': mmm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_mmm_same).fit()
print(result.params)
print(result.summary())

df_ama_same = pd.DataFrame({'Next': ama_ret_all, 'Same': ama_same_all})

result = sm.ols(formula='Next ~ Same', data=df_ama_same).fit()
print(result.params)
print(result.summary())

df_amm_same = pd.DataFrame({'Next': amm_ret_all, 'Same': amm_same_all})

result = sm.ols(formula='Next ~ Same', data=df_amm_same).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare mc top and market w/o mc vs next month market
# Last has high p value (>0.4)

df_lma_mc_womc = pd.DataFrame({'All': lma_ret_all, 'Memecoin': lma_ret_meme, 'AwoMC': lma_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_lma_mc_womc).fit()
print(result.params)
print(result.summary())

df_lmm_mc_womc = pd.DataFrame({'All': lmm_ret_all, 'Memecoin': lmm_ret_meme, 'AwoMC': lmm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_lmm_mc_womc).fit()
print(result.params)
print(result.summary())

df_mma_mc_womc = pd.DataFrame({'All': mma_ret_all, 'Memecoin': mma_ret_meme, 'AwoMC': mma_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_mma_mc_womc).fit()
print(result.params)
print(result.summary())

df_mmm_mc_womc = pd.DataFrame({'All': mmm_ret_all, 'Memecoin': mmm_ret_meme, 'AwoMC': mmm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_mmm_mc_womc).fit()
print(result.params)
print(result.summary())

df_ama_mc_womc = pd.DataFrame({'All': ama_ret_all, 'Memecoin': ama_ret_meme, 'AwoMC': ama_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_ama_mc_womc).fit()
print(result.params)
print(result.summary())

df_amm_mc_womc = pd.DataFrame({'All': amm_ret_all, 'Memecoin': amm_ret_meme, 'AwoMC': amm_ret_awoc})

result = sm.ols(formula='All ~ Memecoin + AwoMC', data=df_amm_mc_womc).fit()
print(result.params)
print(result.summary())

In [ ]:
# @title
# Compare mc top vs next month market w/o mc
# Last has high p value (>0.8)

df_lma_womc = pd.DataFrame({'All': lma_next_awoc, 'Memecoin': lma_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lma_womc).fit()
print(result.params)
print(result.summary())

df_lmm_womc = pd.DataFrame({'All': lmm_next_awoc, 'Memecoin': lmm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_lmm_womc).fit()
print(result.params)
print(result.summary())

df_mma_womc = pd.DataFrame({'All': mma_next_awoc, 'Memecoin': mma_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mma_womc).fit()
print(result.params)
print(result.summary())

df_mmm_womc = pd.DataFrame({'All': mmm_next_awoc, 'Memecoin': mmm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_mmm_womc).fit()
print(result.params)
print(result.summary())

df_ama_womc = pd.DataFrame({'All': ama_next_awoc, 'Memecoin': ama_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_ama_womc).fit()
print(result.params)
print(result.summary())

df_amm_womc = pd.DataFrame({'All': amm_next_awoc, 'Memecoin': amm_ret_meme})

result = sm.ols(formula='All ~ Memecoin', data=df_amm_womc).fit()
print(result.params)
print(result.summary())

In [27]:
# Get Expected performance for each category based on average of 2020-

lw_exp_perf = {}
for cat, df in last_weekly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    lw_exp_perf[cat] = exp

mw_exp_perf = {}
for cat, df in median_weekly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    mw_exp_perf[cat] = exp

aw_exp_perf = {}
for cat, df in mean_weekly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    aw_exp_perf[cat] = exp

lm_exp_perf = {}
for cat, df in last_weekly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    lm_exp_perf[cat] = exp

mm_exp_perf = {}
for cat, df in median_monthly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    mm_exp_perf[cat] = exp

am_exp_perf = {}
for cat, df in mean_monthly_comb.items():
    exp = {}
    for col in df.columns:
        exp[col] = df[col].mean()
    am_exp_perf[cat] = exp




In [28]:
lw_all = last_weekly_comb['All']
lw_meme = last_weekly_comb['Memecoin']
mw_all = median_weekly_comb['All']
mw_meme = median_weekly_comb['Memecoin']
aw_all = mean_weekly_comb['All']
aw_meme = mean_weekly_comb['Memecoin']

lm_all = last_monthly_comb['All']
lm_meme = last_monthly_comb['Memecoin']
mm_all = median_monthly_comb['All']
mm_meme = median_monthly_comb['Memecoin']
am_all = mean_monthly_comb['All']
am_meme = mean_monthly_comb['Memecoin']

lwa_exp_all = lw_exp_perf['All']['Mean']
lwm_exp_all = lw_exp_perf['All']['Median']
lwa_exp_meme = lw_exp_perf['Memecoin']['Mean']
lwm_exp_meme = lw_exp_perf['Memecoin']['Median']

mwa_exp_all = mw_exp_perf['All']['Mean']
mwm_exp_all = mw_exp_perf['All']['Median']
mwa_exp_meme = mw_exp_perf['Memecoin']['Mean']
mwm_exp_meme = mw_exp_perf['Memecoin']['Median']

awa_exp_all = aw_exp_perf['All']['Mean']
awm_exp_all = aw_exp_perf['All']['Median']
awa_exp_meme = aw_exp_perf['Memecoin']['Mean']
awm_exp_meme = aw_exp_perf['Memecoin']['Median']

lma_exp_all = lm_exp_perf['All']['Mean']
lmm_exp_all = lm_exp_perf['All']['Median']
lma_exp_meme = lm_exp_perf['Memecoin']['Mean']
lmm_exp_meme = lm_exp_perf['Memecoin']['Median']

mma_exp_all = mm_exp_perf['All']['Mean']
mmm_exp_all = mm_exp_perf['All']['Median']
mma_exp_meme = mm_exp_perf['Memecoin']['Mean']
mmm_exp_meme = mm_exp_perf['Memecoin']['Median']

ama_exp_all = am_exp_perf['All']['Mean']
amm_exp_all = am_exp_perf['All']['Median']
ama_exp_meme = am_exp_perf['Memecoin']['Mean']
amm_exp_meme = am_exp_perf['Memecoin']['Median']

In [29]:
def pct_change(a, b):
    return (a-b)/abs(b) * 100

lw_epd_all = {}
lw_epd_meme = {}
mw_epd_all = {}
mw_epd_meme = {}
aw_epd_all = {}
aw_epd_meme = {}

lm_epd_all = {}
lm_epd_meme = {}
mm_epd_all = {}
mm_epd_meme = {}
am_epd_all = {}
am_epd_meme = {}


for row in lw_all.itertuples():
    new_row = {'Mean': pct_change(row[1], lwa_exp_all), 'Median': pct_change(row[2], lwm_exp_all)}
    lw_epd_all[row[0]] = new_row
lw_epd_all = pd.DataFrame.from_dict(lw_epd_all, orient='index')
for row in lw_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], lwa_exp_meme), 'Median': pct_change(row[2], lwm_exp_meme)}
    lw_epd_meme[row[0]] = new_row
lw_epd_meme = pd.DataFrame.from_dict(lw_epd_meme, orient='index')

for row in mw_all.itertuples():
    new_row = {'Mean': pct_change(row[1], mwa_exp_all), 'Median': pct_change(row[2], mwm_exp_all)}
    mw_epd_all[row[0]] = new_row
mw_epd_all = pd.DataFrame.from_dict(mw_epd_all, orient='index')
for row in mw_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], mwa_exp_meme), 'Median': pct_change(row[2], mwm_exp_meme)}
    mw_epd_meme[row[0]] = new_row
mw_epd_meme = pd.DataFrame.from_dict(mw_epd_meme, orient='index')

for row in aw_all.itertuples():
    new_row = {'Mean': pct_change(row[1], awa_exp_all), 'Median': pct_change(row[2], awm_exp_all)}
    aw_epd_all[row[0]] = new_row
aw_epd_all = pd.DataFrame.from_dict(aw_epd_all, orient='index')
for row in aw_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], awa_exp_meme), 'Median': pct_change(row[2], awm_exp_meme)}
    aw_epd_meme[row[0]] = new_row
aw_epd_meme = pd.DataFrame.from_dict(aw_epd_meme, orient='index')


for row in lm_all.itertuples():
    new_row = {'Mean': pct_change(row[1], lma_exp_all), 'Median': pct_change(row[2], lmm_exp_all)}
    lm_epd_all[row[0]] = new_row
lm_epd_all = pd.DataFrame.from_dict(lm_epd_all, orient='index')
for row in lm_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], lma_exp_meme), 'Median': pct_change(row[2], lmm_exp_meme)}
    lm_epd_meme[row[0]] = new_row
lm_epd_meme = pd.DataFrame.from_dict(lm_epd_meme, orient='index')

for row in mm_all.itertuples():
    new_row = {'Mean': pct_change(row[1], mma_exp_all), 'Median': pct_change(row[2], mmm_exp_all)}
    mm_epd_all[row[0]] = new_row
mm_epd_all = pd.DataFrame.from_dict(mm_epd_all, orient='index')
for row in mm_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], mma_exp_meme), 'Median': pct_change(row[2], mmm_exp_meme)}
    mm_epd_meme[row[0]] = new_row
mm_epd_meme = pd.DataFrame.from_dict(mm_epd_meme, orient='index')

for row in am_all.itertuples():
    new_row = {'Mean': pct_change(row[1], ama_exp_all), 'Median': pct_change(row[2], amm_exp_all)}
    am_epd_all[row[0]] = new_row
am_epd_all = pd.DataFrame.from_dict(am_epd_all, orient='index')
for row in am_meme.itertuples():
    new_row = {'Mean': pct_change(row[1], ama_exp_meme), 'Median': pct_change(row[2], amm_exp_meme)}
    am_epd_meme[row[0]] = new_row
am_epd_meme = pd.DataFrame.from_dict(am_epd_meme, orient='index')

In [30]:
# Compare mc top/exp vs next week market/exp

df_lwa_epd = pd.DataFrame({'All': lw_epd_all['Mean'], 'Memecoin': lw_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_lwa_epd).fit()
print(result.params)
print(result.summary())

df_lwm_epd = pd.DataFrame({'All': lw_epd_all['Median'], 'Memecoin': lw_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_lwm_epd).fit()
print(result.params)
print(result.summary())

df_mwa_epd = pd.DataFrame({'All': mw_epd_all['Mean'], 'Memecoin': mw_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_mwa_epd).fit()
print(result.params)
print(result.summary())

df_mwm_epd = pd.DataFrame({'All': mw_epd_all['Median'], 'Memecoin': mw_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_mwm_epd).fit()
print(result.params)
print(result.summary())

df_awa_epd = pd.DataFrame({'All': aw_epd_all['Mean'], 'Memecoin': aw_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_awa_epd).fit()
print(result.params)
print(result.summary())

df_awm_epd = pd.DataFrame({'All': aw_epd_all['Median'], 'Memecoin': aw_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_awm_epd).fit()
print(result.params)
print(result.summary())

Intercept    2.708944e-14
Memecoin     1.063480e-01
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    All   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.967
Date:                Mon, 25 Mar 2024   Prob (F-statistic):             0.0269
Time:                        06:52:11   Log-Likelihood:                -1677.8
No. Observations:                 219   AIC:                             3360.
Df Residuals:                     217   BIC:                             3366.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

In [31]:
# Compare mc top/exp vs next month market/exp


df_lma_epd = pd.DataFrame({'All': lm_epd_all['Mean'], 'Memecoin': lm_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_lma_epd).fit()
print(result.params)
print(result.summary())

df_lmm_epd = pd.DataFrame({'All': lm_epd_all['Median'], 'Memecoin': lm_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_lmm_epd).fit()
print(result.params)
print(result.summary())

df_mma_epd = pd.DataFrame({'All': mm_epd_all['Mean'], 'Memecoin': mm_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_mma_epd).fit()
print(result.params)
print(result.summary())

df_mmm_epd = pd.DataFrame({'All': mm_epd_all['Median'], 'Memecoin': mm_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_mmm_epd).fit()
print(result.params)
print(result.summary())

df_ama_epd = pd.DataFrame({'All': am_epd_all['Mean'], 'Memecoin': am_epd_meme['Mean']})

result = sm.ols(formula='All ~ Memecoin', data=df_ama_epd).fit()
print(result.params)
print(result.summary())

df_amm_epd = pd.DataFrame({'All': am_epd_all['Median'], 'Memecoin': am_epd_meme['Median']})

result = sm.ols(formula='All ~ Memecoin', data=df_amm_epd).fit()
print(result.params)
print(result.summary())

Intercept    177.760697
Memecoin       0.181925
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                    All   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     10.93
Date:                Mon, 25 Mar 2024   Prob (F-statistic):            0.00178
Time:                        06:52:12   Log-Likelihood:                -417.21
No. Observations:                  51   AIC:                             838.4
Df Residuals:                      49   BIC:                             842.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------